<a href="https://colab.research.google.com/github/brepowell/ML-Contest-Series/blob/main/MLSeriesSupervisedLearningTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To run:
Use an IDE that will allow you to open a Jupyter Notebook.

For example, use Anaconda Navigator to open Visual Studio Code.

You may need to select a kernel to run the program.

## Import Libraries

In [17]:
# This step can be done in any cell of the notebook. It does not have to be at the top.
import pandas as pd

## Data Information

From:
https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households

More Info:
https://innovation.ukpowernetworks.co.uk/projects/low-carbon-london

"Created 9 years ago, updated 2 years ago
Energy consumption readings for a sample of 5,567 London Households that took part in the UK Power Networks led Low Carbon London project between November 2011 and February 2014.

Readings were taken at half hourly intervals. The customers in the trial were recruited as a balanced sample representative of the Greater London population.

The dataset contains energy consumption, in kWh (per half hour), unique household identifier, date and time. The CSV file is around 10GB when unzipped and contains around 167million rows.

Within the data set are two groups of customers. The first is a sub-group, of approximately 1100 customers, who were subjected to Dynamic Time of Use (dToU) energy prices throughout the 2013 calendar year period. The tariff prices were given a day ahead via the Smart Meter IHD (In Home Display) or text message to mobile phone. Customers were issued High (67.20p/kWh), Low (3.99p/kWh) or normal (11.76p/kWh) price signals and the times of day these applied. The dates/times and the price signal schedule is availaible as part of this dataset. All non-Time of Use customers were on a flat rate tariff of 14.228pence/kWh.

The signals given were designed to be representative of the types of signal that may be used in the future to manage both high renewable generation (supply following) operation and also test the potential to use high price signals to reduce stress on local distribution grids during periods of stress.

The remaining sample of approximately 4500 customers energy consumption readings were not subject to the dToU tariff."


## Create a huge dataframe with all hourly info

In [18]:
import pandas as pd
import glob

def readAllHourlyData():
    # Read all CSV files and concatenate them into a single DataFrame
    path = "data/archive/halfhourly_dataset/halfhourly_dataset/" 

    all_files = glob.glob(path + "block_*.csv")
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)
    full_df = pd.concat(df_list)

    return full_df

londonData = readAllHourlyData()
londonData.shape

C:\Users\brely\AppData\Local\Temp\ipykernel_39724\774332354.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_39724\774332354.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_39724\774332354.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_39724\774332354.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_39724\774332354.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppDa

(167817021, 3)

## Select Features

In [19]:
# What features do I want?
# Blackout signature - mark hours before 0 power as signature days?
# How many customers were represented on that day?
# 
# Weather Features

## Visualize the features

In [20]:
#https://www.tensorflow.org/tutorials/structured_data/time_series

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

londonDataSortedByDay
df = londonDataSortedByDay[5::17520]

df.head()

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
1435,MAC000150,2011-11-24,0.158,0.204,0.158,0.149,0.165,0.135,0.137,0.094,...,0.222,0.376,1.069,0.350,0.671,0.434,0.711,0.198,0.228,0.228
23891,MAC004919,2012-01-22,0.098,0.051,0.032,0.066,0.026,0.025,0.062,0.039,...,0.549,0.461,0.309,0.310,0.073,0.095,0.086,0.063,0.088,0.050
19711,MAC005041,2012-02-19,0.392,5.556,5.432,5.393,4.893,4.648,4.161,2.961,...,0.405,0.431,0.457,0.393,0.365,0.392,0.407,0.354,0.991,0.921
3987,MAC000214,2012-03-11,0.096,0.062,0.044,0.096,0.066,0.043,0.091,0.069,...,0.295,0.253,0.182,0.169,0.265,0.305,0.294,0.285,0.221,0.218
22271,MAC004391,2012-03-28,0.343,0.169,0.190,0.175,0.123,0.183,0.142,0.295,...,0.607,0.488,0.256,0.401,0.804,0.671,0.956,0.475,0.176,0.260


In [21]:
fft = tf.signal.rfft(df.columns[2:])

f_per_dataset = np.arange(0, len(fft))

n_samples_h = len(df.columns[2:])
hh_per_year = 48*365.2524
years_per_dataset = n_samples_h/(hh_per_year)

f_per_year = f_per_dataset/years_per_dataset
plt.step(f_per_year, np.abs(fft))
plt.xscale('log')
plt.ylim(0, 400000)
plt.xlim([0.1, max(plt.xlim())])
plt.xticks([1, 365.2524], labels=['1/Year', '1/day'])
_ = plt.xlabel('Frequency (log scale)')

ValueError: RFFT requires tf.float32 or tf.float64 inputs, got: tf.Tensor(
[b'hh_0' b'hh_1' b'hh_2' b'hh_3' b'hh_4' b'hh_5' b'hh_6' b'hh_7' b'hh_8'
 b'hh_9' b'hh_10' b'hh_11' b'hh_12' b'hh_13' b'hh_14' b'hh_15' b'hh_16'
 b'hh_17' b'hh_18' b'hh_19' b'hh_20' b'hh_21' b'hh_22' b'hh_23' b'hh_24'
 b'hh_25' b'hh_26' b'hh_27' b'hh_28' b'hh_29' b'hh_30' b'hh_31' b'hh_32'
 b'hh_33' b'hh_34' b'hh_35' b'hh_36' b'hh_37' b'hh_38' b'hh_39' b'hh_40'
 b'hh_41' b'hh_42' b'hh_43' b'hh_44' b'hh_45' b'hh_46' b'hh_47'], shape=(48,), dtype=string)

# STEP 2: FEATURE ENGINEERING / DATA CLEANING / PRE-PROCESSING TECHNIQUES

## Label your target variable

In [ ]:
# Ex) Change "yes" or "no" to 1 or 0 so the model understands the label

## Fix the data imbalance and other problems from STEP 1

## Seperate features (x) from labels (y)

## Feature Reduction

## Normalize / Scale the Data

## Investigate Variance or Feature Importance

# STEP 3: MODEL TRAINING & BUILDING


## Split into Train and Test data

## Use a Model

## Perform a Hyperparameter Search

## Save the best model

# STEP 4: EVALUATE THE MODEL

## Look at Metrics - Ex) Precision, Recall, F1 score

## Plot a Confusion Matrix

# STEP 5: DEPLOY MODEL